# KAIM, 2018

## Wasserstein GAN with Gradient Penalty

Inspired from : https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan_gp/wgan_gp.py

In [ ]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch
print("PyTorch Version", torch.__version__)

In [6]:
os.makedirs("images", exist_ok=True)

# parser = argparse.ArgumentParser()
# parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
# parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
# parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
# parser.add_argument("--channels", type=int, default=1, help="number of image channels")
# parser.add_argument("--n_critic", type=int, default=5, help="number of training steps for discriminator per iter")
# parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
# parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
# opt = parser.parse_args()
# print(opt)

n_epochs = 200
batch_size=64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 100
img_size=28
channels = 1
n_critic = 5
clip_value = 0.01
sample_interval = 400


img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False


In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity


In [8]:
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    ),
    batch_size=batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


In [9]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [12]:
# ----------
#  Training
# ----------

batches_done = 0
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

            if batches_done % sample_interval == 0:
                save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += n_critic

[Epoch 0/200] [Batch 0/938] [D loss: 8.033238] [G loss: -0.020339]
[Epoch 0/200] [Batch 5/938] [D loss: 3.967002] [G loss: -0.020647]
[Epoch 0/200] [Batch 10/938] [D loss: -5.078591] [G loss: -0.033391]
[Epoch 0/200] [Batch 15/938] [D loss: -18.109949] [G loss: -0.070267]
[Epoch 0/200] [Batch 20/938] [D loss: -28.971794] [G loss: -0.132993]
[Epoch 0/200] [Batch 25/938] [D loss: -33.419319] [G loss: -0.206448]
[Epoch 0/200] [Batch 30/938] [D loss: -34.980030] [G loss: -0.271374]
[Epoch 0/200] [Batch 35/938] [D loss: -34.448357] [G loss: -0.332536]
[Epoch 0/200] [Batch 40/938] [D loss: -35.855007] [G loss: -0.420371]
[Epoch 0/200] [Batch 45/938] [D loss: -34.929855] [G loss: -0.519464]
[Epoch 0/200] [Batch 50/938] [D loss: -34.520210] [G loss: -0.635209]
[Epoch 0/200] [Batch 55/938] [D loss: -34.684559] [G loss: -0.753071]
[Epoch 0/200] [Batch 60/938] [D loss: -34.269081] [G loss: -0.886731]
[Epoch 0/200] [Batch 65/938] [D loss: -34.331581] [G loss: -1.031770]
[Epoch 0/200] [Batch 70/938

[Epoch 0/200] [Batch 600/938] [D loss: -4.529356] [G loss: -2.717441]
[Epoch 0/200] [Batch 605/938] [D loss: -4.634597] [G loss: -4.071319]
[Epoch 0/200] [Batch 610/938] [D loss: -4.702641] [G loss: -4.455042]
[Epoch 0/200] [Batch 615/938] [D loss: -4.559926] [G loss: -1.391516]
[Epoch 0/200] [Batch 620/938] [D loss: -4.423621] [G loss: -3.485978]
[Epoch 0/200] [Batch 625/938] [D loss: -4.532115] [G loss: -3.540468]
[Epoch 0/200] [Batch 630/938] [D loss: -4.501779] [G loss: -3.410905]
[Epoch 0/200] [Batch 635/938] [D loss: -4.785562] [G loss: -6.342029]
[Epoch 0/200] [Batch 640/938] [D loss: -4.449127] [G loss: -2.745240]
[Epoch 0/200] [Batch 645/938] [D loss: -4.972539] [G loss: -3.580091]
[Epoch 0/200] [Batch 650/938] [D loss: -4.840393] [G loss: -4.745075]
[Epoch 0/200] [Batch 655/938] [D loss: -4.528799] [G loss: -2.963819]
[Epoch 0/200] [Batch 660/938] [D loss: -5.311520] [G loss: -4.826162]
[Epoch 0/200] [Batch 665/938] [D loss: -5.444514] [G loss: -1.730937]
[Epoch 0/200] [Batch

[Epoch 1/200] [Batch 260/938] [D loss: -5.534048] [G loss: -1.320699]
[Epoch 1/200] [Batch 265/938] [D loss: -5.612584] [G loss: -2.011117]
[Epoch 1/200] [Batch 270/938] [D loss: -5.560312] [G loss: -1.069315]
[Epoch 1/200] [Batch 275/938] [D loss: -5.603151] [G loss: -2.313948]
[Epoch 1/200] [Batch 280/938] [D loss: -5.458062] [G loss: -2.591766]
[Epoch 1/200] [Batch 285/938] [D loss: -5.397504] [G loss: -1.926558]
[Epoch 1/200] [Batch 290/938] [D loss: -5.972163] [G loss: -2.161645]
[Epoch 1/200] [Batch 295/938] [D loss: -5.444087] [G loss: -1.370918]
[Epoch 1/200] [Batch 300/938] [D loss: -5.386974] [G loss: -1.924852]
[Epoch 1/200] [Batch 305/938] [D loss: -5.865949] [G loss: -2.068257]
[Epoch 1/200] [Batch 310/938] [D loss: -5.925591] [G loss: -1.780970]
[Epoch 1/200] [Batch 315/938] [D loss: -5.781183] [G loss: -1.771667]
[Epoch 1/200] [Batch 320/938] [D loss: -5.544207] [G loss: -0.843060]
[Epoch 1/200] [Batch 325/938] [D loss: -6.023411] [G loss: -1.473982]
[Epoch 1/200] [Batch

[Epoch 1/200] [Batch 860/938] [D loss: -6.815182] [G loss: -1.253885]
[Epoch 1/200] [Batch 865/938] [D loss: -7.312185] [G loss: -0.922512]
[Epoch 1/200] [Batch 870/938] [D loss: -7.453201] [G loss: -0.828060]
[Epoch 1/200] [Batch 875/938] [D loss: -7.200125] [G loss: -1.219278]
[Epoch 1/200] [Batch 880/938] [D loss: -7.632754] [G loss: 0.280315]
[Epoch 1/200] [Batch 885/938] [D loss: -7.306314] [G loss: -0.069870]
[Epoch 1/200] [Batch 890/938] [D loss: -6.760160] [G loss: -1.657738]
[Epoch 1/200] [Batch 895/938] [D loss: -6.828750] [G loss: -1.992735]
[Epoch 1/200] [Batch 900/938] [D loss: -7.574100] [G loss: 1.119759]
[Epoch 1/200] [Batch 905/938] [D loss: -7.565761] [G loss: 0.005886]
[Epoch 1/200] [Batch 910/938] [D loss: -7.161227] [G loss: -0.540249]
[Epoch 1/200] [Batch 915/938] [D loss: -7.372390] [G loss: -0.769710]
[Epoch 1/200] [Batch 920/938] [D loss: -7.121253] [G loss: -0.052525]
[Epoch 1/200] [Batch 925/938] [D loss: -7.807611] [G loss: -1.223383]
[Epoch 1/200] [Batch 93

[Epoch 2/200] [Batch 535/938] [D loss: -7.992206] [G loss: -2.053398]
[Epoch 2/200] [Batch 540/938] [D loss: -8.340628] [G loss: -1.197436]
[Epoch 2/200] [Batch 545/938] [D loss: -8.079260] [G loss: -1.589360]
[Epoch 2/200] [Batch 550/938] [D loss: -8.112549] [G loss: -1.696105]
[Epoch 2/200] [Batch 555/938] [D loss: -7.838699] [G loss: -2.490852]
[Epoch 2/200] [Batch 560/938] [D loss: -7.890328] [G loss: -2.021032]
[Epoch 2/200] [Batch 565/938] [D loss: -8.206180] [G loss: -2.263305]
[Epoch 2/200] [Batch 570/938] [D loss: -8.130158] [G loss: -1.790826]
[Epoch 2/200] [Batch 575/938] [D loss: -7.746278] [G loss: -1.912567]
[Epoch 2/200] [Batch 580/938] [D loss: -8.056869] [G loss: -2.643064]
[Epoch 2/200] [Batch 585/938] [D loss: -7.959714] [G loss: -1.951472]
[Epoch 2/200] [Batch 590/938] [D loss: -8.182262] [G loss: -1.118640]
[Epoch 2/200] [Batch 595/938] [D loss: -8.266701] [G loss: -0.474647]
[Epoch 2/200] [Batch 600/938] [D loss: -8.104960] [G loss: -2.112254]
[Epoch 2/200] [Batch

...

[Epoch 49/200] [Batch 310/938] [D loss: -2.011892] [G loss: -1.815487]
[Epoch 49/200] [Batch 315/938] [D loss: -1.950111] [G loss: -2.171689]
[Epoch 49/200] [Batch 320/938] [D loss: -1.941903] [G loss: -1.711116]
[Epoch 49/200] [Batch 325/938] [D loss: -2.299373] [G loss: -1.572311]
[Epoch 49/200] [Batch 330/938] [D loss: -2.131245] [G loss: -2.089654]
[Epoch 49/200] [Batch 335/938] [D loss: -1.379303] [G loss: -2.043683]
[Epoch 49/200] [Batch 340/938] [D loss: -2.148906] [G loss: -1.818809]
[Epoch 49/200] [Batch 345/938] [D loss: -2.131434] [G loss: -1.563429]
[Epoch 49/200] [Batch 350/938] [D loss: -2.079787] [G loss: -1.361628]
[Epoch 49/200] [Batch 355/938] [D loss: -2.062129] [G loss: -1.397848]
[Epoch 49/200] [Batch 360/938] [D loss: -1.786699] [G loss: -1.907960]
[Epoch 49/200] [Batch 365/938] [D loss: -1.780736] [G loss: -2.061640]
[Epoch 49/200] [Batch 370/938] [D loss: -2.176271] [G loss: -1.972412]
[Epoch 49/200] [Batch 375/938] [D loss: -1.948974] [G loss: -1.950191]
[Epoch

[Epoch 49/200] [Batch 910/938] [D loss: -2.377465] [G loss: -1.473082]
[Epoch 49/200] [Batch 915/938] [D loss: -2.299895] [G loss: -1.638885]
[Epoch 49/200] [Batch 920/938] [D loss: -2.190906] [G loss: -1.967753]
[Epoch 49/200] [Batch 925/938] [D loss: -2.137899] [G loss: -2.124879]
[Epoch 49/200] [Batch 930/938] [D loss: -2.421909] [G loss: -2.016956]
[Epoch 49/200] [Batch 935/938] [D loss: -1.862969] [G loss: -2.802305]
[Epoch 50/200] [Batch 0/938] [D loss: -2.224147] [G loss: -2.613958]
[Epoch 50/200] [Batch 5/938] [D loss: -1.929643] [G loss: -2.296706]
[Epoch 50/200] [Batch 10/938] [D loss: -1.799723] [G loss: -2.154801]
[Epoch 50/200] [Batch 15/938] [D loss: -2.311706] [G loss: -2.017860]
[Epoch 50/200] [Batch 20/938] [D loss: -2.245643] [G loss: -2.437123]
[Epoch 50/200] [Batch 25/938] [D loss: -2.106346] [G loss: -2.200334]
[Epoch 50/200] [Batch 30/938] [D loss: -2.175558] [G loss: -1.615047]
[Epoch 50/200] [Batch 35/938] [D loss: -1.992864] [G loss: -1.817806]
[Epoch 50/200] [

[Epoch 50/200] [Batch 560/938] [D loss: -2.001477] [G loss: -2.256260]
[Epoch 50/200] [Batch 565/938] [D loss: -1.736608] [G loss: -1.967249]
[Epoch 50/200] [Batch 570/938] [D loss: -2.343610] [G loss: -1.541540]
[Epoch 50/200] [Batch 575/938] [D loss: -2.499840] [G loss: -1.760228]
[Epoch 50/200] [Batch 580/938] [D loss: -1.817951] [G loss: -1.648449]
[Epoch 50/200] [Batch 585/938] [D loss: -2.142858] [G loss: -1.524410]
[Epoch 50/200] [Batch 590/938] [D loss: -1.805226] [G loss: -2.330319]
[Epoch 50/200] [Batch 595/938] [D loss: -2.021127] [G loss: -1.944687]
[Epoch 50/200] [Batch 600/938] [D loss: -1.987748] [G loss: -1.536339]
[Epoch 50/200] [Batch 605/938] [D loss: -1.788506] [G loss: -1.747717]
[Epoch 50/200] [Batch 610/938] [D loss: -1.907445] [G loss: -2.397353]
[Epoch 50/200] [Batch 615/938] [D loss: -1.722895] [G loss: -2.060131]
[Epoch 50/200] [Batch 620/938] [D loss: -1.867293] [G loss: -1.909848]
[Epoch 50/200] [Batch 625/938] [D loss: -1.986308] [G loss: -1.731132]
[Epoch

[Epoch 51/200] [Batch 200/938] [D loss: -2.157785] [G loss: -1.748349]
[Epoch 51/200] [Batch 205/938] [D loss: -1.970119] [G loss: -1.757149]
[Epoch 51/200] [Batch 210/938] [D loss: -2.066015] [G loss: -1.457874]
[Epoch 51/200] [Batch 215/938] [D loss: -2.041576] [G loss: -1.615868]
[Epoch 51/200] [Batch 220/938] [D loss: -1.850708] [G loss: -1.308533]
[Epoch 51/200] [Batch 225/938] [D loss: -1.974037] [G loss: -1.229804]
[Epoch 51/200] [Batch 230/938] [D loss: -2.314063] [G loss: -1.241021]
[Epoch 51/200] [Batch 235/938] [D loss: -2.136529] [G loss: -1.335193]
[Epoch 51/200] [Batch 240/938] [D loss: -1.987619] [G loss: -1.972156]
[Epoch 51/200] [Batch 245/938] [D loss: -2.246897] [G loss: -1.816119]
[Epoch 51/200] [Batch 250/938] [D loss: -2.092494] [G loss: -1.662479]
[Epoch 51/200] [Batch 255/938] [D loss: -2.548966] [G loss: -1.080404]
[Epoch 51/200] [Batch 260/938] [D loss: -2.154745] [G loss: -1.421340]
[Epoch 51/200] [Batch 265/938] [D loss: -1.722083] [G loss: -1.716760]
[Epoch

KeyboardInterrupt: 